In [2]:
import pandas as pd
import numpy as np
us_banks = pd.read_csv('data/usbanks_locations.csv')

/var/folders/qj/gh_j11514m37mqtfrlr885k40000gn/T/ipykernel_64556/1841898131.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  us_banks = pd.read_csv('data/usbanks_locations.csv')


## Creating data to control for any commercial banking

In [3]:
len(us_banks[us_banks['COUNTY'].isin(['Bronx', 'Kings', 'New York', 'Queens', 'Richmond'])])

1571

In [4]:
nyc_banks = us_banks[us_banks['COUNTY'].isin(['Bronx', 'Kings', 'New York', 'Queens', 'Richmond'])]

In [5]:
def format(address, city, zip):
    address = address.replace(" ", "+")
    city = city.replace(" ", "+")
    return address + '%2C+' + city + '%2C+NY%2C+' + str(zip)

def format_row_address(row):
    return format(row['ADDRESS'],row['CITY'],row['ZIP'])

def format_normal_address(row):
    return row['ADDRESS'] + ', ' + row['CITY'] + ', NY, ' + str(row['ZIP'])

formatted_rows_column = nyc_banks.apply(lambda x: format_row_address(x), axis=1)
formatted_rows_normal = nyc_banks.apply(lambda x: format_normal_address(x), axis=1)

In [6]:
formatted_rows_normal.iloc[30]

'31-81 Steinway Street, Astoria, NY, 11103'

In [7]:
nyc_banks['formatted_address'] = formatted_rows_normal

/var/folders/qj/gh_j11514m37mqtfrlr885k40000gn/T/ipykernel_64556/4134371361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_banks['formatted_address'] = formatted_rows_normal


In [8]:
import requests

# location given here
address = formatted_rows_normal.iloc[30]

def lookup_tract(address):
    # api-endpoint
    URL = "https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?"
    
    # defining a params dict for the parameters to be sent to the API
    PARAMS = {'address': address,
            'benchmark':'2020',
            'vintage':'2010',
            'format':'json'}
    
    # sending get request and saving the response as response object
    r = requests.get(url = URL, params = PARAMS)
    
    # extracting data in json format
    data = r.json()

    # return tract
    try:
        return data['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['TRACT'],data['result']['addressMatches'][0]['geographies']['County Subdivisions'][0]['BASENAME']
    except:
        return '-1','-1'

In [ ]:
def associate_tract_with_address(row):
    return lookup_tract(row['formatted_address'])

tracts = nyc_banks.apply(lambda x: associate_tract_with_address(x), axis=1)

In [ ]:
nyc_banks['tract'] = tracts

/var/folders/qj/gh_j11514m37mqtfrlr885k40000gn/T/ipykernel_47248/110791871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_banks['tract'] = tracts


In [ ]:
nyc_banks.to_csv('nyc_banks_subboro.csv')